## TOM SCRAPER
Author: Jude Darmanin
Date: December 2023

This code scrapes all articles from timesofmalta.com from the chosen category and stores them in an SQL database. Was written as part of my Univerity of London degree.

In [1]:
import requests
from bs4 import BeautifulSoup
import json 
from time import sleep
import itertools
from datetime import datetime as dt
import sqlite3


In [2]:
#SET VARIABLES
header = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}

category = 'national' #select which category to scrape

f_url = 'tom_urls.txt'

db_TOM = 'articles_TOM.db'

##### SCRAPE ARTICLE URLS

In [ ]:
#scrape all URLs
urls = []

for i in itertools.count(start=1):
    try:
        html = requests.get(f'https://timesofmalta.com/articles/listing/{category}/page:{i}', headers = header).content
        soup = BeautifulSoup(html,"html.parser")
        section = soup.find('script', id = 'listing-ld')
        listings = json.loads(section.contents[0])['@graph'] #script data is in json format; 'contents' returns content within HTML tags. @graph object contains listing metadata.
        for l in listings:
            url = l['url']
            urls.append(url)
        sleep(0.1)
    except:
        print(f'Script stopped at page {i}')
        break
    
#save to txt file
with open(f_url, 'a') as f:
    for u in urls:
        f.write(f"{u}\n")

##### SCRAPE AND STORE ARTICLES

In [ ]:
#CREATE TABLE/DB to store scraped data
sql_create_table = """CREATE TABLE IF NOT EXISTS articles (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    title TEXT,
                    date DATE,
                    article TEXT,
                    keywords TEXT,
                    author TEXT,
                    publisher TEXT,
                    url TEXT,
                    main_img_url TEXT,
                    main_img_cap TEXT);"""


conn = sqlite3.connect(db_TOM)
cursor = conn.cursor()
# Create a table to store your articles
cursor.execute(sql_create_table)

conn.commit()
conn.close()

def store_data(db, data:dict):
    '''Function to store data in db'''

    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO articles (title, date, article, keywords, author, publisher, url, main_img_url, main_img_cap)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (data['title'], data['date'], data['article'], data['keywords'], data['author'], data['publisher'], data['url'], data['main_img_url'], data['main_img_cap']))

    conn.commit()
    conn.close()

In [5]:
#SCRAPING ARTICLES 

#read urls
with open(f_url, 'r') as f:
    rurls = f.readlines()

#scraper
errors = []
for counter, u in enumerate(rurls): 

    try:
        html=BeautifulSoup(requests.get(u).content,"html.parser")
        script = html.find('script',{'id':'article-ld'}) 
        content = json.loads(script.contents[0])['@graph'][0]
        
        data = {}

        data['title'] = content['headline']
        data['publisher'] = content['publisher']['@id']
        data['keywords'] = content['keywords']
        data['date'] = dt.strptime(content['datePublished'], '%Y-%m-%dT%H:%M:%S%z').strftime('%Y-%m-%d')
        data['author'] = content['author'][0]['name']
        data['article'] = content['articleBody']
        data['main_img_url'] = content['image'][0]['url']
        data['main_img_cap'] = content['image'][0]['caption']
        data['url'] = u

        store_data(db_TOM, data)
    except:
        errors.append(u)
        continue


##### READING DATA

In [6]:
def get_column_names(db, table_name):
    '''Get table column names'''
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    #use PRAGMA to get column names
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns_info = cursor.fetchall()

    conn.close()

    #extract column names 
    column_names = [column_info[1] for column_info in columns_info]

    return column_names


def get_all_articles(db):
    '''Read articles from DB'''
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM articles')
    articles = cursor.fetchall()

    column_names = get_column_names(db, 'articles')
    articles = [dict(zip(column_names, article)) for article in articles]

    conn.close()

    return articles

#reading
all_articles = get_all_articles(db_TOM)